In [1]:
%run -i '/home/hadoop/.Turing/TuringAllFunctions.py'

------------- Turing Lab Data Science --------------------
------ Funções carregadas em memória com sucesso ---------
--------- Data da última atualização: --------------------
--------------- 13/05/2019 -------------------------------
----------------------------------------------------------
--- Consulte o manual de Biblioteca de Funções Turing: ---


----------------------------------------------------------


In [2]:
#---------- Definição dos Parâmetros do projeto ---------------------------------
nm_bucket = 'turing-bkt-treinamentos'
id_empresa = '1022'

# Id do projeto parametrizado no Turing (o mesmo valor dado ao campo Id Projeto do Timeline)
id_projeto = '350002'

# Nome da variável resposta (quando houve)
targetname = 'Survived'

# Delimitador da tabela 
abt_delimiter = ','

In [3]:
################### Não alterar este parágrafo #####################################
#---------- Gerando Credenciais para Acesso Seguro ---------------------------------
id_modelo = id_projeto
schema = 'EMP_'+ id_empresa +'_Proj_'+ id_projeto
S3fs,S3session,S3client,S3resource = TuringUsersCredentialsControl(id_empresa)
schema_rds = 'EMP_'+id_empresa+'_Proj_'+id_projeto
mysq_conn,engine = TuringRDSManagementConn(id_empresa,schema_rds,turing_adm = 'N')
path_s3_wfile = 's3://'+nm_bucket+'/Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'
s3_path = path_s3_wfile
####################################################################################

#### Leitura da tabela desenvolvimento (holdout) escorada pelo Turing 

In [42]:
nm_file_desenv_holdout = 'scored_best_model_desenv.csv'
path_ = 'Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'
path_file = path_+str(nm_file_desenv_holdout)
desenv_df_ho = TuringReadS3CSVFile(nm_bucket,path_file,sep=abt_delimiter)

desenv_df_ho.rename( columns={'Unnamed: 0':'Id'}, inplace=True )
desenv_df_ho.shape

(485, 10)

In [43]:
desenv_df_ho.head()

,Id,Fare,Age,Pclass,TGD_Sex_female,TGD_Sex_male,P_Target_0,P_Target_1,C_Target,target_t_bin
0,11,-0.516566,0.000000,0.848448,1,0,0.390056,0.609944,1,0
1,496,-0.196153,0.313981,-0.342054,1,0,0.229696,0.770304,1,1
2,468,0.498508,0.000000,0.848448,0,1,0.880521,0.119479,0,0
3,424,-0.512419,0.000000,0.848448,1,0,0.390064,0.609936,1,1
4,576,-0.455387,0.467718,-0.342054,0,1,0.783886,0.216114,0,0


#### Leitura da tabela validaçao (holdout) escorada pelo Turing 

In [44]:
nm_file_valid_holdout = 'scored_best_model_valid.csv'
path_ = 'Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'
path_file = path_+str(nm_file_valid_holdout)
valid_df_ho = TuringReadS3CSVFile(nm_bucket,path_file,sep=abt_delimiter)

valid_df_ho.rename( columns={'Unnamed: 0':'Id'}, inplace=True )
valid_df_ho.shape

(138, 10)

In [45]:
valid_df_ho.head()

,Id,Fare,Age,Pclass,TGD_Sex_female,TGD_Sex_male,P_Target_0,P_Target_1,C_Target,target_t_bin
0,313,-0.339250,1.082665,0.848448,0,1,0.919588,0.080412,0,0
1,387,-0.326807,-0.454703,0.848448,1,0,0.346879,0.653121,1,1
2,214,-0.403540,0.313981,-0.342054,1,0,0.229431,0.770569,1,1
3,60,-0.283515,-1.453991,0.848448,0,1,0.801148,0.198852,0,0
4,343,-0.417020,0.006508,-0.342054,1,0,0.207818,0.792182,1,1


#### Calculando KS - Desenvolvimento

In [46]:
KS_df,KS_value_d = TuringEvaluateModels(desenv_df_ho,'P_Target_0','target_t_bin')
KS_df.head(10)

,min_scr,max_scr,bads,goods,total,odds,bad_rate,ks,max_ks
0,0.043221,0.155921,47,2,49,0.04,95.92%,23.93,
1,0.156570,0.282200,40,8,48,0.20,83.33%,42.15,
2,0.285217,0.390696,35,16,51,0.46,68.63%,55.03,<== KS ==
3,0.390733,0.558055,16,30,46,1.88,34.78%,53.20,
4,0.558267,0.725778,16,33,49,2.06,32.65%,50.36,
5,0.730648,0.791576,8,40,48,5.00,16.67%,40.94,
6,0.799587,0.850655,6,42,48,7.00,12.50%,29.79,
7,0.850658,0.879742,10,39,49,3.90,20.41%,21.77,
8,0.879749,0.881697,6,42,48,7.00,12.50%,10.62,
9,0.882500,0.956879,7,42,49,6.00,14.29%,0.00,


#### Calculando KS - Validação

In [47]:
KS_df,KS_value_v = TuringEvaluateModels(valid_df_ho,'P_Target_0','target_t_bin')
KS_df.head(10)

,min_scr,max_scr,bads,goods,total,odds,bad_rate,ks,max_ks
0,0.064337,0.152380,14,0,14,0.00,100.00%,27.45,
1,0.156570,0.229634,13,1,14,0.08,92.86%,51.79,
2,0.229949,0.361324,7,7,14,1.00,50.00%,57.47,
3,0.390059,0.528062,6,7,13,1.17,46.15%,61.19,
4,0.534825,0.672508,7,7,14,1.00,50.00%,66.87,<== KS ==
5,0.674505,0.783950,2,12,14,6.00,14.29%,57.00,
6,0.786943,0.842423,1,12,13,12.00,7.69%,45.17,
7,0.842478,0.874000,1,13,14,13.00,7.14%,32.18,
8,0.875762,0.880037,0,14,14,inf,0.00%,16.09,
9,0.880727,0.967211,0,14,14,inf,0.00%,0.00,


#### Calculando Gini - Desenvolvimento

In [49]:
gini_desenv = TuringGini(desenv_df_ho,'target_t_bin','P_Target_1')
gini_desenv*100

65.66

#### Calculando Gini - Validação

In [50]:
gini_valid = TuringGini(valid_df_ho,'target_t_bin','P_Target_1')
gini_valid*100

82.28

#### Consolidando Metricas 

In [51]:
metricas = [['Desenv', KS_value_d,gini_desenv], ['Valid', KS_value_v,gini_valid]]
df = pd.DataFrame(metricas, columns = ['ABT','KS', 'Gini']) 
df

,ABT,KS,Gini
0,Desenv,55.03,0.6566
1,Valid,66.87,0.8228
